In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd
import random
import math
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, OrdinalEncoder, MinMaxScaler
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from tqdm import tqdm

def encoding_categorical_variables(data):

    all_cols = data.columns.tolist()

    encoder = OrdinalEncoder(categories=[['at_home', 'teacher', 'health','services','other']])
    data[['Mjob']] = encoder.fit_transform(data[['Mjob']]).astype('int64')
    data[['Fjob']] = encoder.fit_transform(data[['Fjob']]).astype('int64')

    encoder = OrdinalEncoder(categories=[['other', 'mother', 'father']])
    data[['guardian']] = encoder.fit_transform(data[['guardian']]).astype('int64')

    encoder = OrdinalEncoder(categories=[['home', 'reputation', 'course', 'other']])
    data[['reason']] = encoder.fit_transform(data[['reason']]).astype('int64')

    encoder = OrdinalEncoder(categories=[['no', 'yes']])
    data[['schoolsup']] = encoder.fit_transform(data[['schoolsup']]).astype('int64')
    data[['famsup']] = encoder.fit_transform(data[['famsup']]).astype('int64')
    data[['paid']] = encoder.fit_transform(data[['paid']]).astype('int64')
    data[['activities']] = encoder.fit_transform(data[['activities']]).astype('int64')
    data[['nursery']] = encoder.fit_transform(data[['nursery']]).astype('int64')
    data[['higher']] = encoder.fit_transform(data[['higher']]).astype('int64')
    data[['internet']] = encoder.fit_transform(data[['internet']]).astype('int64')
    data[['romantic']] = encoder.fit_transform(data[['romantic']]).astype('int64')

    categorical_cols = ['school', 'sex', 'address', 'famsize', 'Pstatus']
    categorical_data = data[categorical_cols]

    encoder = OneHotEncoder(sparse_output=False)
    encoded_data = encoder.fit_transform(categorical_data).astype('int64')

    data_encoded = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(categorical_cols))

    numeric_cols = [col for col in all_cols if col not in categorical_cols]

    data = pd.concat([data_encoded, data[numeric_cols]], axis=1)

    # Droping column 'age' as G3 scores are independent of 'age'
    data.drop('age', axis=1, inplace=True)

    return data

def target_variable_encoding(data, data_test):

  # For binary classification tasks, according to the research paper, pass is considered if G3 >= 10, else fail
    for index, row in data.iterrows():
        if row['G3'] >= 13:
            data.at[index, 'G3'] = 1
        else:
            data.at[index, 'G3'] = 0

    for index, row in data_test.iterrows():
        if row['G3'] >= 13:
            data_test.at[index, 'G3'] = 1
        else:
            data_test.at[index, 'G3'] = 0

    return [data, data_test]

def test_train_split(data):
    # Out of 649 samples, 120 random samples are considered for the test set, and remaining ones for training.
    test_data_indices = random.sample(range(650), 120)

    data_test = data.iloc[test_data_indices].copy()
    data.drop(index=test_data_indices, inplace=True)

    data_test.reset_index(drop=True, inplace=True)
    data.reset_index(drop=True, inplace=True)

    return [data, data_test]

def feature_scaling(data, data_test):

  # All these columns are considered for feature scaling, to bring their value between 0 & 1
    feature_scaling_cols = ['Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures',
                          'famrel','freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences', 'G1', 'G2']

    scaler = MinMaxScaler()

    data[feature_scaling_cols] = scaler.fit_transform(data[feature_scaling_cols])

    data_test[feature_scaling_cols] = scaler.transform(data_test[feature_scaling_cols])

    return [data, data_test]

def train_test_split_arrays(data, data_test):

    X_train = data.iloc[:, :-1].values

    # The target variable variable 'G3' is considered as 'y'
    y_train = data.iloc[:, -1:].values


    X_test = data_test.iloc[:, :-1].values
    y_test = data_test.iloc[:, -1:].values

    return [X_train, y_train, X_test, y_test]

In [ ]:
data = pd.read_csv('student-por.csv', sep=",")
data = encoding_categorical_variables(data)
data, data_test = test_train_split(data)
data, data_test = feature_scaling(data, data_test)
data, data_test = target_variable_encoding(data, data_test)
x_train, y_train, x_test, y_test = train_test_split_arrays(data, data_test)
x = np.vstack((x_train, x_test))
y = np.vstack((y_train, y_test))

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2)
model = LogisticRegression()
model.fit(x_train, y_train.ravel())

In [ ]:
# To get max Delta(x,h) such that h is consistent with h_hat on T
def find_h1(h_hat, T, x):

    lag_terms = []
    lambs = np.linspace(0,40,num=100)
    present_model = None

    for j in range(len(lambs)):
        new_model = LogisticRegression(max_iter = 500)
        yi = []
        lamb = lambs[j]
        weights = []

        g0 = g1 = 0
        for i in range(len(x)):
            g0 += x[i,2]== 0
            g1 += x[i,2]==1

        for i in range(len(x)):
            Ci0 = (x[i,2] == 1) * 0 + (x[i,2] == 0) * (-1/g0)
            Ci1 = (x[i,2] == 1) * (-1/g1) + (x[i,2] == 0) * 0

            if i in T:
                Ci0 += -lamb * (h_hat.predict(x[i].reshape(1,-1))[0] == 0)
                Ci1 += -lamb * (h_hat.predict(x[i].reshape(1,-1))[0] == 1)

            weights.append(np.abs(Ci0 - Ci1))
            yi.append(Ci0 >= Ci1)

        new_model.fit(x, yi,sample_weight=weights)
        present_model = new_model

        lag_term = 0
        for s in T:
            lag_term += new_model.predict(x[s].reshape(1,-1)) == h_hat.predict(x[s].reshape(1,-1))
        lag_terms.append(lag_term)
        if(lag_term == len(T)):
            break

    return present_model

In [ ]:
# To get min Delta(x,h) such that h is consistent with h_hat on T
def find_h2(h_hat, T, x):

    lag_terms = []
    lambs = np.linspace(0,40,num=100)
    present_model = None

    for j in range(len(lambs)):
        new_model = LogisticRegression(max_iter=500)
        yi = []
        lamb = lambs[j]
        weights = []
        g0 = g1 = 0
        for i in range(len(x)):
            g0 += x[i,2]== 0
            g1 += x[i,2]==1
        for i in range(len(x)):
            Ci0 = (x[i,2] == 1) * 0 + (x[i,2] == 0) * (1/g0)
            Ci1 = (x[i,2] == 1) * (1/g1) + (x[i,2] == 0) * 0

            if i in T:
                Ci0 += -lamb * (h_hat.predict(x[i].reshape(1,-1))[0] == 0)
                Ci1 += -lamb * (h_hat.predict(x[i].reshape(1,-1))[0] == 1)

            weights.append(np.abs(Ci0 - Ci1))
            yi.append(Ci0 >= Ci1)

        new_model.fit(x, yi,sample_weight=weights)
        present_model = new_model
        lag_term = 0
        for s in T:
            lag_term += new_model.predict(x[s].reshape(1,-1)) == h_hat.predict(x[s].reshape(1,-1))
        lag_terms.append(lag_term)

        if(lag_term == len(T)):
            break

    return present_model

In [ ]:
def online_learning_oracle(X,predictions,S):
    model = LogisticRegression()
    y = []
    for i in S:
        y.append(predictions[i])
    y = np.array(y)
    if(len(S) > 0):
        model.fit(X[S], y.ravel())
    else:
        y = np.array([0,1]).reshape(-1,1)
        model.fit(X[[0,1]], y.ravel())
    return model

def mu(model, X):
    num_0 = 0
    num_1 = 0
    sum_0 = 0
    sum_1 = 0

    for i in range(len(X)):
        if(X[i,2]):
            num_1 += 1
            sum_1 += model.predict(X[i].reshape(1,-1))[0] == 1

        else:
            num_0 += 1
            sum_0 += model.predict(X[i].reshape(1,-1))[0] == 1

    if(num_0 > 0 and num_1 > 0): return sum_1/num_1 - sum_0/num_0
    elif(num_0 == 0): return sum_1/num_1
    return sum_0/num_0

In [ ]:
# Main Active Fairness Auditing Algorithm
def active_auditing(X,M,model,epsilon,budget):
    
    S = set()
    predictions = dict()
    budget_used = 0

    while True:
        w = [1/len(X) for _ in range(len(X))]
        fail_prob = 0.1
        tau_x = [np.random.exponential(1/(2*np.log(X.shape[1]) + np.log(M) - np.log(fail_prob))) for _ in range(len(X))]
        h_hat = online_learning_oracle(X,predictions,list(S))  
        T = set()

        while True:
            h1 = find_h1(h_hat, T, X)
            h2 = find_h2(h_hat, T, X)

            mu_diff = mu(h1,X) - mu(h2,X)
  
            if mu_diff <= 2 * epsilon:
                break  

            delta = {i for i in range(len(X)) if h1.predict(X[i].reshape(1,-1)) != h_hat.predict(X[i].reshape(1,-1)) or h2.predict(X[i].reshape(1,-1)) != h_hat.predict(X[i].reshape(1,-1))}

            while sum(w[i] for i in delta) <= 1 + 1e-3:
                for i in delta:
                    w[i] *= 2  
                T = set([i for i in range(len(x)) if w[i] >= tau_x[i]])

        diff = 0
        for el in T:
            diff += h_hat.predict(X[el].reshape(1,-1)) != model.predict(X[el].reshape(1,-1))

        
        for i in T:
            if(budget_used >= budget): break
            predictions[i] = model.predict(X[i].reshape(1,-1))[0]
            S.add(i)
            budget_used += 1

        if(budget_used >= budget):
            return S

In [ ]:
# Run this multiple times to get the mean value of the deviations
budgets = 50
M = 100
epsilon = 0.25
u_D = np.abs(mu(model, x))

S = list(active_auditing(x, M, model, epsilon, budget))
u1 = np.abs(mu(find_h1(model, S, x), x))
u2 = np.abs(mu(find_h2(model, S, x), x))
u = max(u1, u2)
u_S = np.abs(mu(model, x[S]))
print(max(u - u_D, 0))